# EDA of Heart Failure Data

Joel, Yaniv, Jeff<br>
Merck Challenge - Project 2<br>
Rutgers MSDS<br>
2/4/21<br>

Items to Check for EDA:  
1. Data loading and checking  
2a. Numerical summaries  
2b. Graphical summaries  
3. Extract important variable(s) and analyze them  
3a. Check for outliers and missing data  
(Note: Did not include a on outliers (yet), as should determine the threshold of removing outliers first. I would suggest checking the jackknife residuals (refer to HW #6 from reg & TS))(EDIT: might be different for GLMs...)  
3b. Develop and test simple models  
3c. Test model assumptions

Q: How much EDA should we do? (e.g. doing summaries of a few vs all variables, using some vs all of Yaniv's EDA code, etc.)

In [1]:
### Preliminaries
import pandas as pd # essential for data analysis
import numpy as np # gives tools for working with arrays
import matplotlib.pyplot as plt # useful for plots in general
import seaborn as sns # needed for histograms
import pylab as pyl #for generating plots: show()
import statsmodels.api as sm  #to use the Logit()

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# Loading and Checking Data
!pwd
data = pd.read_csv("/Users/yanivbronshtein/Downloads/heart_failure_clinical_records_dataset.csv")  # type and enter pwd and see if dataset is in same directory
data.head()

In [ ]:
data.describe()

In [ ]:
data.shape # 299 rows, 13 cols

In [ ]:
data.std()

In [ ]:
data.info() # info about each variable

In [ ]:
data.isnull().any() # check if any missing data
# Q: Which variable(s) is dtype referring to?

In [ ]:
## Correlation matrix
data.corr()

In [ ]:
var = 'DEATH_EVENT' # defining and checking descriptive stats of variable of interest
series = data[var]

In [ ]:
print("Mean: {}".format(series.mean()))
print("Stdev: {}".format(series.std()))
print("Min: {}".format(series.min()))
print("Max: {}".format(series.max()))
print("Median: {}".format(series.median()))
print("Mode: {}".format(series.mode()))
print("Skew: {}".format(series.skew()))
print("Kurtosis: {}".format(series.kurtosis()))

In [ ]:
### Graphical Summaries
## Scatterplots
sns.scatterplot(x = "age", y = "DEATH_EVENT", data = data)

In [ ]:
sns.scatterplot(x = "diabetes", y = "DEATH_EVENT", data = data)

In [ ]:
sns.scatterplot(x = "sex", y = "DEATH_EVENT", data = data)

In [ ]:
sns.scatterplot(x = "age", y = "platelets", data = data) # used for testing continous DV

In [ ]:
#sns.pairplot(data = data)
# Q: How to zoom in on particular plots?

In [ ]:
## Boxplots
sns.boxplot(x='sex', y='DEATH_EVENT', data= data)

In [ ]:
sns.boxplot(x='diabetes', y='DEATH_EVENT', data= data)

In [ ]:
sns.boxplot(x='age', y='DEATH_EVENT', data= data)

In [ ]:
# Get variable names

def getvar(x):
    for i in x:
        print(i)

In [ ]:
getvar(data)

In [ ]:
## Histograms
# Note: See cell 22 in Yaniv's EDA notebook

print(min(data['age']))
print(max(data['age'])) # used to check if increasing or decreasing bins is needed
data['age'].hist(bins=30)

In [ ]:
print(min(data['anaemia']))
print(max(data['anaemia'])) # used to check if increasing or decreasing bins is needed
data['anaemia'].hist(bins=30)

In [ ]:
print(min(data['creatinine_phosphokinase']))
print(max(data['creatinine_phosphokinase'])) # used to check if increasing or decreasing bins is needed
data['creatinine_phosphokinase'].hist(bins=30)

In [ ]:
print(min(data['diabetes']))
print(max(data['diabetes'])) # used to check if increasing or decreasing bins is needed
data['diabetes'].hist(bins=30)

In [ ]:
print(min(data['ejection_fraction']))
print(max(data['ejection_fraction'])) # used to check if increasing or decreasing bins is needed
data['ejection_fraction'].hist(bins=30)

In [ ]:
print(min(data['high_blood_pressure']))
print(max(data['high_blood_pressure'])) # used to check if increasing or decreasing bins is needed
data['high_blood_pressure'].hist(bins=30)

In [ ]:
print(min(data['platelets']))
print(max(data['platelets'])) # used to check if increasing or decreasing bins is needed
data['platelets'].hist(bins=30)

In [ ]:
print(min(data['serum_creatinine']))
print(max(data['serum_creatinine'])) # used to check if increasing or decreasing bins is needed
data['serum_creatinine'].hist(bins=30)

In [ ]:
print(min(data['serum_sodium']))
print(max(data['serum_sodium'])) # used to check if increasing or decreasing bins is needed
data['serum_sodium'].hist(bins=30)

In [ ]:
print(min(data['sex']))
print(max(data['sex'])) # used to check if increasing or decreasing bins is needed
data['sex'].hist(bins=30)

In [ ]:
print(min(data['smoking']))
print(max(data['smoking'])) # used to check if increasing or decreasing bins is needed
data['smoking'].hist(bins=30)

In [ ]:
print(min(data['time']))
print(max(data['time'])) # used to check if increasing or decreasing bins is needed
data['time'].hist(bins=30)

In [ ]:
print(min(data['DEATH_EVENT']))
print(max(data['DEATH_EVENT'])) # used to check if increasing or decreasing bins is needed
data['DEATH_EVENT'].hist(bins=30)

In [ ]:
## Outliers
# Note: Can do it qualitatively or quantitatively... will be doing the former here. However, we can take the latter
# approach if we so desired: https://medium.com/datadriveninvestor/finding-outliers-in-dataset-using-python-efc3fce6ce32
# Q: Should use Judgement vs. IQR/Z-scores?

# Ex # 1: platelets variable
# Before
sns.scatterplot(x='platelets', y = 'DEATH_EVENT', data=data)

In [ ]:
# After
# See cell 32 in Yaniv's EDA notebook

data = data[data['platelets'] <= 650000]
sns.scatterplot(x='platelets', y = 'DEATH_EVENT', data=data)

In [ ]:
# Ex #2: age variable
# Note: don't think we should exclude age, as study wanted to focus on all age groups, but just to see

# Before
sns.scatterplot(x='age', y = 'DEATH_EVENT', data=data)

In [ ]:
# After

data = data[data['age'] <= 85]
sns.scatterplot(x='age', y = 'DEATH_EVENT', data=data)

# Q: Which variables to ultimately include in predictive models? Important to know since they may need to be adjusted for outliers

In [ ]:
## Logistic Regression
## NOTE: As DEATH_EVENT is a binary variable, we must do logit; linear regression will not work
## Reference: https://gist.github.com/reenashaw/9de3608fa94f1a9ade0e19135afac70a

data['int']=1 #explicitly create a placeholder for y-intercept: b0
data.head()

In [ ]:
IV = ['age','sex','diabetes']
model= sm.Logit(data['DEATH_EVENT'], data[IV])
answer= model.fit()

In [ ]:
answer.summary() #summarize the model

# (K-Fold) Cross-Validation Logistic Regression

Note: The following functions/commands are based on those from Yaniv's Logistic Regression Notebook

In [ ]:
# Before splitting data

print(data.shape)
data.head()

In [ ]:
# After splitting data and creating training data

train_df = data.sample(int(data.shape[0] * 0.8))
print(train_df.shape)
train_df.head()

In [ ]:
testing_rows = [idx for idx in data.index if idx not in train_df.index]
len(testing_rows)

In [ ]:
# # Creating test data

# test_df = data.iloc[testing_rows, :]
# print(test_df.shape)
# test_df.head()

In [ ]:
# Building logistic regression model

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
def getvar(x):
    for i in x:
        print(i)

print(getvar(data))

In [ ]:
# inlcude ALL numerical variables (thankfully, this data is clean...)

train = train_df[['age','anaemia', 'creatinine_phosphokinase', 'diabetes', 'ejection_fraction', 
                  'high_blood_pressure', 'platelets', 'serum_creatinine', 'serum_sodium', 
                  'sex', 'smoking', 'time', 'DEATH_EVENT']]
train.head()

In [ ]:
# create x and y subsets of train and train data
# note: DEATH_EVENT = response variable (y); dropped in training set?

X_train, X_test, Y_train, Y_test = train_test_split(train.drop('DEATH_EVENT', axis=1), 
                                                    train['DEATH_EVENT'], test_size = 0.20, 
                                                    random_state=101)

In [ ]:
model = LogisticRegression()
model.fit(X_train,Y_train)
predictions = model.predict(X_test)
print(classification_report(Y_test, predictions))
print(accuracy_score(Y_test, predictions)*100, "%")     # convert to %

# Q: Should also produce smaller model (e.g. model with fewer variables), to see if this is improved model?

# Random Forest Classifier

**Step 1: Import libraries**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

**Step 2: Create the set`X` for the feature set and `Y` for the treatment variable from the dataframe** 

In [ ]:
X = data.drop(['time','DEATH_EVENT'], axis=1)

In [ ]:
X.head()

In [ ]:
Y = data['DEATH_EVENT']

In [ ]:
Y.head()

**Create function to perform the following:
a)train, test, split the feature set and treatment set <br>
b). Train a Random Forest Model with 100 trees and criterion entropy<br>
c). Fit the Model<br>
d). Generate the Predictions<br>
e). Create the necessary metric reports**<br>

In [ ]:
import random #Necessary so that the data is split according to a random metric
import collections
from sklearn import metrics

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 101)
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
random_forest_preds = random_forest.predict(X_test)
print('The accuracy of the Random Forests model is :\t',metrics.accuracy_score(random_forest_preds,Y_test))
print(confusion_matrix(Y_test, random_forest_preds))


In [ ]:
### importing LIME modules

import lime
import lime.lime_tabular

**Create a function to display the patients that the model failed to classify correctly**

In [ ]:
def display_anomalies(model_preds):
    index_list = []
    for row_index, (input, prediction, label) in enumerate(zip (X_test, model_preds, Y_test)):
      if prediction != label:
        print('Row', row_index, 'has been classified as ', prediction, 'and should be ', label)
        index_list.append(row_index)
    
    print("************************************************************************************")

    for index in index_list:
        print(X_test.iloc[index,:])
        print("************************************************************************************")



In [ ]:
display_anomalies(random_forest_preds)

In [ ]:
# predicting x based on RFs (explained using lime)

predict_fn_rf = lambda x: random_forest.predict_proba(x).astype(float)
X = X_train.values
explainer = lime.lime_tabular.LimeTabularExplainer(X,feature_names = X_train.columns,class_names=['Died','Survived'],kernel_width=5)

In [ ]:
### Interpret LIME results (note: refer to data description for variable names and info)
# Ex 1:

print(X_test.loc[[38]],"death event is: ",Y_test.loc[[38]])
print("")
print("number of predictors are: ",len(X_test.columns))
choosen_instance = X_test.loc[[38]].values[0]
exp = explainer.explain_instance(choosen_instance, predict_fn_rf,num_features=12)
exp.show_in_notebook(show_all=False)

## Frop top to bottom (right side of middle plot):
## We can see that the follow-up period (days), level of serum creatinine in the blood (mg/dL), 
## percentage of blood leaving the heart at each contraction (percentage), and level of the CPK enzyme in the blood (mcg/L)
## had the most positive influence on the patient's survivability. Considering these features and all others,
## the patient had an 75% chance of surviving.

In [ ]:
df_a = pd.DataFrame(Y_test)
df_b = pd.DataFrame(random_forest_preds)
comparison = pd.DataFrame({"Y_test":Y_test, "random_forest_preds": random_forest_preds})
comparison

In [ ]:
# Ex 2:

print(X_test.loc[[37]],"death event is: ",Y_test.loc[[37]])
print("")
print("number of predictors are: ",len(X_test.columns))
choosen_instance = X_test.loc[[37]].values[0]
exp = explainer.explain_instance(choosen_instance, predict_fn_rf,num_features=12)
exp.show_in_notebook(show_all=False)

## Frop top to bottom (right side of middle plot):
## We can see that the follow-up period (days) and the patient's age had the most positive influence 
## on the patient's survivability. Considering these features and all others, the patient had an 82% chance of surviving.

In [ ]:
# Ex 3:

print(X_test.loc[[213]],"death event is: ",Y_test.loc[[213]])
print("")
print("number of predictors are: ",len(X_test.columns))
choosen_instance = X_test.loc[[213]].values[0]
exp = explainer.explain_instance(choosen_instance, predict_fn_rf,num_features=12)
exp.show_in_notebook(show_all=False)

## Frop top to bottom (right side of middle plot):
## We can see that the follow-up period (days), level of serum creatinine in the blood (mg/dL), 
## percentage of blood leaving the heart at each contraction (percentage), and level of serum sodium in the blood (mEq/L)
## had the most positive influence on the patient's survivability.Considering these features and all others,
## the patient had an 96% chance of surviving.

In [ ]:
# Ex 4:

print(X_test.loc[[53]],"death event is: ",Y_test.loc[[53]])
print("")
print("number of predictors are: ",len(X_test.columns))
choosen_instance = X_test.loc[[53]].values[0]
exp = explainer.explain_instance(choosen_instance, predict_fn_rf,num_features=12)
exp.show_in_notebook(show_all=False)

## Frop top to bottom (right side of middle plot):
## We can see that the follow-up period (days), percentage of blood leaving the heart at each contraction (percentage),
## and the patient's age had the most negative influence on the patient survivability. Considering these features and all others,
## the patient had an 82% chance of dying.

Takeaway:

Based on some of the results above, it would seem that time is the most influential feature on the patient's survivability. This is expected, since, as patients age, they are more likely to experience heart failure, regardless of their gender. Whether they survive such a medical outbreak depends on their physiological makeup - particularly, certain chemical content in their blood, however.

(Note: Would like to interpret more observations, but the code below doesn't work for certain values for some reason... tried 115, 44, 79, etc.)

In [ ]:
 #Actually survived but model said they died
print(X_test.loc[[155]],"death event is: ",Y_test.loc[[155]])
print("")
print("number of predictors are: ",len(X_test.columns))
choosen_instance = X_test.loc[[155]].values[0]
exp = explainer.explain_instance(choosen_instance, predict_fn_rf,num_features=12)
exp.show_in_notebook(show_all=False)

# Note: doesn't work for certain observations... how come? (code was already here)

# Partial Dependence Plots (PDPs)

In [ ]:
feature_names = X_test.columns.tolist()
feature_names

In [ ]:
from matplotlib import pyplot as plt
from pdpbox import pdp, get_dataset, info_plots



In [ ]:
def create_pdp_plot(model, feature_names, feature_name):
    # Create the data that we will plot
    pdp_goals = pdp.pdp_isolate(model=model, dataset=X_test, model_features=feature_names, feature=feature_name)

    # plot it
    pdp.pdp_plot(pdp_goals, feature_name)
    plt.show()


In [ ]:
create_pdp_plot(random_forest, feature_names, feature_names[2])

In [ ]:
create_pdp_plot(random_forest, feature_names, feature_names[4])

In [ ]:
create_pdp_plot(random_forest, feature_names, feature_names[7])

In [ ]:
create_pdp_plot(random_forest, feature_names, feature_names[9])

In [ ]:
create_pdp_plot(random_forest, feature_names, feature_names[10])

# 2D PDP Plots

In [ ]:
def create_pdp_2d(model, feature_names, indices, plot_type):
    # Similar to previous PDP plot except we use pdp_interact instead of pdp_isolate and pdp_interact_plot instead of pdp_isolate_plot
    features_to_plot = [feature_names[i] for i in indices]
    inter1  =  pdp.pdp_interact(model=model, dataset=X_test, model_features=feature_names, features=features_to_plot)

    pdp.pdp_interact_plot(pdp_interact_out=inter1, feature_names=features_to_plot, plot_type=plot_type)
    plt.show()


In [ ]:
create_pdp_2d(random_forest, feature_names, (2,4), 'contour')

In [ ]:
create_pdp_2d(random_forest, feature_names, (2,7), 'contour')

In [ ]:
create_pdp_2d(random_forest, feature_names, (2,9), 'contour')

In [ ]:
create_pdp_2d(random_forest, feature_names, (4,7), 'contour')

In [ ]:
create_pdp_2d(random_forest, feature_names, (4,9), 'contour')

In [ ]:
create_pdp_2d(random_forest, feature_names, (7,9), 'contour')

In [ ]:
create_pdp_2d(random_forest, feature_names, (2,4), 'grid')

In [ ]:
create_pdp_2d(random_forest, feature_names, (2,7), 'grid')

In [ ]:
create_pdp_2d(random_forest, feature_names, (2,9), 'grid')

In [ ]:
create_pdp_2d(random_forest, feature_names, (4,7), 'grid')

In [ ]:
create_pdp_2d(random_forest, feature_names, (4,9), 'grid')

In [ ]:
create_pdp_2d(random_forest, feature_names, (7,9), 'grid')

# Shapley Values